In [2]:
from tkinter import *
from tkinter import messagebox
import random
# import pyperclip
import json

In [7]:
# PASSWORD GENERATOR

def password_generator():
    
    password_entry.delete(0, END)
    
    letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
    numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    symbols = ['!', '#', '$', '%', '&', '(', ')', '*', '+']

    nr_letters = random.randint(8, 10)
    nr_symbols = random.randint(2, 4)
    nr_numbers = random.randint(2, 4)

    password_list = [random.choice(letters) for letter in range(nr_letters)]
    password_list += [random.choice(symbols) for symbol in range(nr_symbols)]
    password_list += [random.choice(numbers) for number in range(nr_numbers)]

    random.shuffle(password_list)

    # password_list has been shuffled
    password = "".join(password_list)
    
    # insert the password inside the password entry
    password_entry.insert(END, password)
    # BUG --> if we press more tha once it generates more passwords all at once
    # at the begeinning we need to delete everything inside the entry field
    # add password_entry.delete(0, END)
    
    # use pyperclip to automatically copy our password
    pyperclip.copy(password)
    
# ----------------------------------------------------------------------------------------

# SAVE PASSWORD
def take_inputs():
    """ this function takes three inputs and store them in a text file.
    It also deletes the inputs from the entry field"""
    web_input = website_entry.get()
    email_input = username_email_entry.get()
    password_input = password_entry.get()
    
    # create a nested dictionary
    # web_input is the first level key (what we're going to be searching through)
    # global new_data
    # this file is overwritten any time we call it --> we need to store each separate block inside a json document
    new_data = {
        web_input:{
            "email": email_input,
            "pw": password_input}
    }
    
    # CHECK FOR EMPTY FIELDS
    if len(web_input) == 0 or len(password_input) == 0:
        
        messagebox.showwarning(title="empty fileds message", message="don't leave empty fields")
    else:
        
        # POP-UP MESSAGE
        is_ok = messagebox.askokcancel(title=website,message=f"details entered: {email_input} and {password_input}\n is it ok to save?")
        # <class 'bool'>
        # ADD INPUTS TO THE DOCUMENT
        if is_ok:
            try:
                with open("./data.json", mode="r") as file:
                    data = json.load(file)
                    data.update(new_data)
            except FileNotFoundError:
                with open("./data.json", mode="w") as file:
                    json.dump(new_data, file, indent=4)
            else:                
                with open("./data.json", mode="w") as file:
                    json.dump(data, file, indent=4)
                    
            # count how many items we have inside the document
            items_count()
                
            # DELATE THE ENTRY INPUTS
            website_entry.delete(0,END)
            # username_email_entry.delete(0,END)
            password_entry.delete(0,END)

            
def find_password():
    # this is what the user is looking for
    user_entry = website_entry.get()
    
    try:
        # open the json file
        with open("./data.json") as file:
            data = json.load(file)
            
    except FileNotFoundError:
        messagebox.showinfo(title="Search", message="No data file found")

    else:
        if user_entry.capitalize() in data:
            search_info = data[user_entry.capitalize()]
            email_info = search_info.get("email")
            pw_info = search_info.get("pw")
            messagebox.showinfo(title="Info", message=f"email: {email_info}\npassword: {pw_info}")
            
        else:
            messagebox.showwarning(title="Search", message="No details for the website found")

    finally:
        last_item_searched()
        website_entry.delete(0,END)
        

def last_item_searched():
    # take input from entry
    last_item_input = website_entry.get().capitalize()
    # delete last entry
    last_search_entry.delete(0,END)
    # insert recent input in entry
    last_search_entry.insert(0,last_item_input)
    
    # BUG 5: it doesn't show the string but Null! --> SOLVED
    # this overwrote the document so when we called it the string was
    # empty because we didn't have an input in the website
    with open("./search.json", "w") as data_search:
        json.dump(last_item_input, data_search, indent=2)
    # BUG 6: what if we don't have a search.json file --> handling errors!
    
# BUG 4: if site is added as lowercase and we search it inside our json file it tells us "item not found"

def remove_collection():
    # BUG 1: the message pops up even without entry --> SOLVED
    # BUG 2: it doesn't remove the item from the data.json doc but only from the dictionary --> SOLVED
    # BUG 3: no error handling defined if the document is empty --> SOLVED
    item_to_remove = website_entry.get().capitalize()
    try:
        with open("./data.json") as file:
            data = json.load(file)
    except FileNotFoundError:
        messagebox.showinfo(title="info message",message="your document is empty or does not exist\nadd a new site and password")
    else:
        if len(item_to_remove) == 0 or item_to_remove not in data:
            messagebox.showinfo(title="remove item",message="there's no item to remove")
        else:
            remove = messagebox.askyesno(title="remove item", message=f"are you sure you want to remove {item_to_remove} password and email?")
            if remove:
                # we want remove the file from the json document, not from the dictionary, which will always have an item
                # data = json.load(file)
                # now that we save the content in "data" we can delete an item
                data.pop(item_to_remove)
                with open("./data.json", "w") as file:
                    json.dump(data, file, indent=4)
                # update the number of items
                items_count()
                
                    
def items_count():
    with open("./data.json") as file:
        data = json.load(file)
        items = len(data)
        total_item.config(text=f"total item in store: {items}")

BG = "#2C394B"
ALTCOL = "#40E0D0"
DEFAULT = "#F0F0F0"

def switch(color=BG):
    # change background color
    window.config(bg=color)
    # change button and background
    toggle_on.config(image=toggle_off_img, background=color, activebackground=color, command=lambda: switch_back(color=DEFAULT))
    # change canvas background
    canvas.config(bg=color, highlightthickness=0)
    # change labels' foreground and background
    # BUG 7: if we take this out of the function it doesn't work proprely
    labels = [website, last_search, username_email, password, total_item]
    for label in labels:
        label.config(bg=color, foreground="white")

# we could have just copied everything in switch and just change colors --> but we can use arguments in command!
def switch_back(color=DEFAULT):
    switch(color=color)
    toggle_on.config(image=toggle_on_img, background=color, activebackground=color, command=lambda: switch(color=BG))
    labels = [website, last_search, username_email, password, total_item]
    for label in labels:
        label.config(bg=color, foreground="black")

In [8]:
# LAYOUT
window = Tk()
window.title("Password Manager")
window.config(pady=50,padx=50)

# create canvas
canvas = Canvas(width=200, height=200)
# call image
canvas_img = PhotoImage(file="./key-lock-64.png")
canvas.create_image(100,100, image=canvas_img,)

canvas.grid(column=1,row=0)

website = Label(text="Website:")
website.grid(column=0,row=1)
website_entry = Entry(width=32)
website.focus()
website_entry.grid(column=1,row=1)


with open("./search.json", "r") as data_search:
    last_item = json.load(data_search)
last_search = Label(text="Last search")
last_search.grid(column=0,row=2)
last_search_entry = Entry(width=32)
last_search_entry.insert(0,last_item)
last_search_entry.grid(column=1, row=2)


username_email = Label(text="Email/Username:")
username_email.grid(column=0,row=3)
username_email_entry = Entry(width=50)
username_email_entry.insert(END, "mvlchess@gmail.com")
username_email_entry.grid(column=1,row=3, columnspan=2)


password = Label(text="Password:")
password.grid(column=0,row=4)
password_entry = Entry(width=32)
password_entry.grid(column=1,row=4, columnspan=1)


gen_password = Button(text="Generate Password", width=14, command=password_generator)
gen_password.grid(column=2,row=4)


add_password = Button(text="Add", width=42, command=take_inputs)
add_password.grid(column=1,row=5, columnspan=2)


remove_button = Button(text="Remove", width=42, command=remove_collection)
remove_button.grid(column=1,row=6, columnspan=2, pady=8)


search_button = Button(text="Search", width=13, command=find_password)
search_button.grid(column=2, row=1)

# ADD TOTAL ITEM IN DATA.JSON FILE
total_item = Label(text="")
total_item.grid(column=0, row=7)
items_count()


toggle_on_img = PhotoImage(file="./switch-on.png")
toggle_off_img = PhotoImage(file="./switch-off.png")
# use argument tkinter command function
# https://stackoverflow.com/questions/6920302/how-to-pass-arguments-to-a-button-command-in-tkinter
toggle_on = Button(image=toggle_on_img, highlightthickness=0, borderwidth=0, command=lambda: switch(color=BG))
toggle_on.place(x=410,y=0)


window.mainloop()